In [2]:
!pip install pandas tensorflow scikit-learn transformers matplotlib

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import TFBertModel, AutoTokenizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# google colab
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Dataset_PeduliPasal/dataset_pidana_umum.csv")

# local computer
# df = pd.read_csv("../data/dataset_csv/dataset_pidana_umum.csv")

In [4]:
df.head()

,amar,amar_lainnya,id,klasifikasi,lama_hukuman,lembaga_peradilan,provinsi,status,sub_klasifikasi,url,...,identitas,riwayat_penahanan,riwayat_perkara,riwayat_tuntutan,riwayat_dakwaan,fakta,amar_putusan,penutup,fakta_hukum,pertimbangan_hukum
0,pidana,hukum,00035681c8d944203f25d2e8215ae2bf,pidana-umum,210,pn-kudus,jateng,berkekuatan-hukum-tetap,pemalsuan,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap eny sulistiyaningsih binti mashad...,terdakwa ditahan dengan jenis tahanan rutan se...,pengadilan negeri tersebutsetelah membaca berk...,setelah mendengar tuntutan requsitoir penuntut...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa selanjutnya untuk membuktikan ...,mengadili 1 menyatakan terdakwa eny sulistiyan...,demikian diputuskan dalam rapat permusyawarata...,NaN,NaN
1,pidana,hukum,000399ce26773e18695ce14f519cb9e6,pidana-umum,720,pn-demak,jateng,berkekuatan-hukum-tetap,pencurian,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ali maftuhin bin nur salim tempat...,terdakwa ditahan di rumah tahanan negara berda...,pengadilan negeri tersebutsetelah membacapenet...,setelah mendengar surat tuntutan pidana requis...,menimbang bahwa terdakwa didakwa oleh penuntut...,menimbang bahwa untuk menguatkan dakwaan terse...,mengadili 1 menyatakan terdakwa ali maftuhin b...,demikianlah diputuskan dalam rapat permusyawar...,menimbang bahwa berdasarkan keterangan saksi s...,NaN
2,pidana,jatuh-pidana-oleh-karena-itu-kepada-dakwa-ir-b...,0006582ad67cd9bd1ddf4261a09bf382,pidana-umum,120,pn-kediri,jatim,berkekuatan-hukum-tetap,kejahatan-terhadap-keamanan-negara,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ir bambang sasongko bin r soewarn...,terdakwa tidak ditahan,terdakwa didampingi oleh penasehat hukumnya ya...,telah mendengar pembacaan tuntutan pidana oleh...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa selanjutnya dipersidangan tela...,mengadili1 menyatakan terdakwa ir bambang saso...,demikian diputuskan dalam rapat musyawarah maj...,NaN,NaN
3,pidana,hukum,00122b1be15a10ad474bb3b7ec0dea73,pidana-umum,90,pn-sampang,jatim,berkekuatan-hukum-tetap,penghinaan,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ahmad al pak saki tempat lahir sa...,NaN,terdakwa dipersidangan tidak didampingi oleh p...,telah mendengar tuntutan pidana dari penuntut ...,menimbang bahwa berdasarkan catatan penuntut u...,menimbang bahwa dalam persidangan telah dideng...,mengadili 1 menyatakan terdakwa ahmad al pak s...,demikian diputuskan pada hari senin tanggal 5 ...,menimbang bahwa berdasarkan keterangan saksi s...,NaN
4,pidana,hukum,00136d1554e18c63256deac42aad0c58,pidana-umum,210,pn-cirebon,jabar,berkekuatan-hukum-tetap,pencurian,https://putusan3.mahkamahagung.go.id/direktori...,...,1 nama lengkap muhamad rizki als rizki bin edi...,terdakwa ditangkap pada tanggal juli 2019terda...,terdakwa tidak didampingi penasihat hukumpenga...,setelah mendengar pembacaan tuntutan pidana ya...,menimbang bahwa terdakwa diajukan ke persidang...,menimbang bahwa untuk membuktikan dakwaannya p...,mengadili1 menyatakan terdakwa muhamad rizki a...,demikian diputuskan dalam sidang permusyawarat...,menimbang bahwa berdasarkan alat bukti dan bar...,menimbang bahwa selanjutnya majelis hakim akan...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10729 entries, 0 to 10728
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   amar                10729 non-null  object
 1   amar_lainnya        10729 non-null  object
 2   id                  10729 non-null  object
 3   klasifikasi         10729 non-null  object
 4   lama_hukuman        10729 non-null  int64 
 5   lembaga_peradilan   10729 non-null  object
 6   provinsi            10729 non-null  object
 7   status              10729 non-null  object
 8   sub_klasifikasi     10729 non-null  object
 9   url                 10729 non-null  object
 10  kepala_putusan      10727 non-null  object
 11  identitas           9481 non-null   object
 12  riwayat_penahanan   8830 non-null   object
 13  riwayat_perkara     10630 non-null  object
 14  riwayat_tuntutan    10228 non-null  object
 15  riwayat_dakwaan     10577 non-null  object
 16  fakta               10

In [7]:
# Features and label
features = ['riwayat_dakwaan', 'amar_putusan', 'fakta']
label = 'sub_klasifikasi'

# Drop miss value
df = df.dropna(subset=features + [label])

# Combine textual features into a single column
df['combined_text'] = df[features].fillna('').agg(' '.join, axis=1)

# Encode Label
sub_classification = df[label].unique()

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(sub_classification)
num_classes = len(label_encoder.classes_)

print("Encoded Labels:", labels_encoded)
print("Number of Classes:", num_classes)

Encoded Labels: [ 9 14  3 17 13 18 24  6 16 15  7 19 20  5 10  0 12  2 22 11  8 23  4  1
 21]
Number of Classes: 25


In [8]:
MODEL_NAME = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = TFBertModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to

In [ ]:
MAX_LENGTH = 128

# Fungsi untuk memproses data dengan tokenizer
def encode_texts(texts, tokenizer, max_length=MAX_LENGTH):
    return tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

X_encoded = encode_texts(list(df['combined_text']), tokenizer)
input_ids = encode_texts["input_ids"]
attention_mask = encode_texts["attention_mask"]

In [ ]:
# Split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
def create_model(max_length=MAX_LENGTH):
    # Input layer
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

    # Pre-trained IndoBERT
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    cls_token = bert_output.last_hidden_state[:, 0, :]  # Ambil representasi token [CLS]

    # Fully Connected Layers untuk klasifikasi
    x = tf.keras.layers.Dense(256, activation="relu")(cls_token)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    # Definisikan model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    return model

In [ ]:
model = create_model()

In [ ]:
# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Train Model
model.fit(
    [input_ids, attention_mask],
    y_train,
    epochs=3,
    batch_size=2
)

In [ ]:
# Evaluasi Model
loss, accuracy = model.evaluate(
    x={
        "input_ids": test_encodings['input_ids'],
        "attention_mask": test_encodings['attention_mask']
    },
    y=y_test
)
print(f"Accuracy: {accuracy}")

In [ ]:
# Prediksi pada Data Baru
predictions = model.predict({
    "input_ids": test_encodings['input_ids'][:5],
    "attention_mask": test_encodings['attention_mask'][:5]
})
predicted_classes = label_encoder.inverse_transform(predictions.argmax(axis=1))
print("Predicted Categories:", predicted_classes)

In [ ]:
example_text = "Menimbang, bahwa terdakwa..."
tokenized_example = tokenizer(example_text, max_length=200, padding="max_length", truncation=True)
print(tokenized_example)